# **Projet 5 :** Segmentez des clients d'un site e-commerce
Votre mission est d’aider les équipes d’Olist à comprendre les différents types d'utilisateurs. Vous utiliserez donc des méthodes non supervisées pour regrouper ensemble des clients de profils similaires. Ces catégories pourront être utilisées par l’équipe marketing pour mieux communiquer.

# <span style="color:red">**Sommaire**</span><a name="tableofcontents"></a>

1. [Importations](#importations)
2. [Jointures](#join)
3. [Transtypage](#transtype)
4. [Sélection des variables](#selection)
5. [Ingénierie des caractéristiques (achats)](#engineering)
6. [Observations dupliquées, valeurs manquantes](#missing)
7. [Aggrégation des données](#aggregation)
8. [Ingénierie des caractéristiques (clients)](#engineering_customers)
9. [Encodage](#encoding)
10. [Profiling](#profiling)
11. [Sauvegarde](#save)

# [[⇑]](#tableofcontents) **Importations** des données et modules <a name="importations"></a>

In [1]:
# Installation : pip install flake8 pycodestyle pycodestyle_magic
# Activate %%flake8 magic function
%load_ext pycodestyle_magic

In [2]:
# %%flake8  # checking PEP8 compliance
import pandas as pd
import functions.functions_p5 as functions_p5

# import importlib
# importlib.reload(functions_p5)

In [3]:
# %%flake8  # checking PEP8 compliance
import os
from IPython.display import display

dir_name = 'data'
dict_df = {}

# Importing .csv files as pd.DataFrame
for file_name in os.listdir(dir_name):
    if file_name[-4:] == '.csv':
        file_path = os.path.join(dir_name, file_name)
        dict_df[file_name[:-4]] = pd.read_csv(file_path)

# Displays name and head of each DataFrame
for name, data_df in dict_df.items():
    print(name, "\n")
    display(data_df.head(2))

olist_customers_dataset 



,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP


olist_geolocation_dataset 



,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP


olist_orders_dataset 



...

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0


olist_sellers_dataset 



,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP


product_category_name_translation 



,product_category_name,product_category_name_english
0,beleza_saude,health_beauty
1,informatica_acessorios,computers_accessories


Nous constatons que cette base de données contient des données personnelles pseudonymisées (mais pas anonymisées), qui permettraient potentiellement d'identifier les consommateurs. Dans l'UE, l'usage de telles données est encadré par le RGPD (*Règlement Général sur la Protection des Données*).

# [[⇑]](#tableofcontents) **Jointures des tables** <a name="join"></a>
Les données sont présentées dans une base de données relationnelle sous forme normale. Nous nous intéressons à la segmentation des clients. Nous allons donc faire les jointures de données nécessaires sur les clients pour avoir toutes les données dans une table unique.

### Jointure des tables 'customers_dataset' et 'orders_dataset'
Pour associer les cilents aux commandes. Nous faisons une jointure extérieure "gauche" pour conserver les données des clients qui ne sont pas associés à une commande.

In [4]:
# %%flake8  # checking PEP8 compliance

# Dataframes to join
data = dict_df['olist_customers_dataset']
df2 = dict_df['olist_orders_dataset']

# Joining dataframe, specifying the column to merge on
data = pd.merge(data, df2, on='customer_id', how="left")  # left join

## Jointure avec la table 'order_payments_dataset'

In [5]:
# %%flake8  # checking PEP8 compliance

# Dataframe to join
df2 = dict_df['olist_order_payments_dataset']

# Joining dataframe, specifying the column to merge on
data = pd.merge(data, df2, on='order_id', how="left")  # left join

## Jointure avec la table 'order_reviews_dataset'

In [6]:
# %%flake8  # checking PEP8 compliance

# Dataframe to join
df2 = dict_df['olist_order_reviews_dataset']

# Joining dataframe, specifying the column to merge on
data = pd.merge(data, df2, on='order_id', how="left")  # left join

## Jointure avec la table 'order_item_dataset'

In [7]:
# %%flake8  # checking PEP8 compliance

# Dataframe to join
df2 = dict_df['olist_order_items_dataset']

# Joining dataframe, specifying the column to merge on
data = pd.merge(data, df2, on='order_id', how="left")  # left join

## Jointure avec la table 'products_dataset'

In [8]:
# %%flake8  # checking PEP8 compliance

# Dataframe to join
df2 = dict_df['olist_products_dataset']

# Joining dataframe, specifying the column to merge on
data = pd.merge(data, df2, on='product_id', how="left")  # left join

## Jointure avec la table 'translation_dataset'

In [9]:
# %%flake8  # checking PEP8 compliance

# Dataframe to join
df2 = dict_df['product_category_name_translation']

# Joining dataframe, specifying the column to merge on
data = pd.merge(data, df2, on='product_category_name', how="left")  # left join

## Jointure avec la table 'sellers_dataset'

In [10]:
# %%flake8  # checking PEP8 compliance

# Dataframes to join
df2 = dict_df['olist_sellers_dataset']

# Joining dataframe, specifying the column to merge on
data = pd.merge(data, df2, on='seller_id', how="left")  # left join

## Jointure avec la table 'geolocalisation_dataset' (pour les clients)

TO DO : Moyenner les latitudes et longitudes pour un même code postal.

In [11]:
# %%flake8  # checking PEP8 compliance

# Dataframes to join
df1 = data
df2 = dict_df['olist_geolocation_dataset']

# Drop different localisation for the same 'geolocation_zip_code_prefix'
df2 = df2.drop_duplicates(subset='geolocation_zip_code_prefix')

# Joining dataframe, specifying the column to merge on
data = pd.merge(
    data,
    df2,
    left_on='customer_zip_code_prefix',
    right_on='geolocation_zip_code_prefix',
    how="left")  # left join

In [12]:
data.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,...,product_width_cm,product_category_name_english,seller_zip_code_prefix,seller_city,seller_state,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05-16 15:05:35,2017-05-16 15:22:12,2017-05-23 10:47:57,...,31.0,office_furniture,8577.0,itaquaquecetuba,SP,14409.0,-20.509897,-47.397866,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,29150127e6685892b6eab3eec79f59c7,delivered,2018-01-12 20:48:24,2018-01-12 20:58:32,2018-01-15 17:14:59,...,40.0,housewares,88303.0,itajai,SC,9790.0,-23.726853,-46.545746,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,2018-05-19 16:07:45,2018-05-20 16:19:10,2018-06-11 14:31:00,...,17.0,office_furniture,8577.0,itaquaquecetuba,SP,1151.0,-23.527788,-46.660310,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP,951670f92359f4fe4a63112aa7306eba,delivered,2018-03-13 16:06:38,2018-03-13 17:29:19,2018-03-27 23:22:42,...,28.0,office_furniture,8577.0,itaquaquecetuba,SP,8775.0,-23.496930,-46.185352,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,2018-07-29 09:51:30,2018-07-29 10:10:09,2018-07-30 15:16:00,...,35.0,home_confort,14940.0,ibitinga,SP,13056.0,-22.987222,-47.151073,campinas,SP


# [[⇑]](#tableofcontents) **Transtypage des données** <a name="transtype"></a>

In [13]:
# Downcasting the dtypes of numerical columns
# help(functions_p5.downcast_dtypes)
functions_p5.downcast_dtypes(data);

Column customer_zip_code_prefix downcasted from int64 to int32.
Column review_score downcasted from int64 to int8.
Column payment_sequential downcasted from float64 to float32.
Column payment_installments downcasted from float64 to float32.
Column payment_value downcasted from float64 to float32.
Column order_item_id downcasted from float64 to float32.
Column price downcasted from float64 to float32.
Column freight_value downcasted from float64 to float32.
Column product_name_lenght downcasted from float64 to float32.
Column product_description_lenght downcasted from float64 to float32.
Column product_photos_qty downcasted from float64 to float32.
Column product_weight_g downcasted from float64 to float32.
Column product_length_cm downcasted from float64 to float32.
Column product_height_cm downcasted from float64 to float32.
Column product_width_cm downcasted from float64 to float32.
Column seller_zip_code_prefix downcasted from float64 to float32.
Column geolocation_zip_code_prefix d

{'customer_zip_code_prefix': 'int32',
 'review_score': 'int8',
 'payment_sequential': 'float32',
 'payment_installments': 'float32',
 'payment_value': 'float32',
 'order_item_id': 'float32',
 'price': 'float32',
 'freight_value': 'float32',
 'product_name_lenght': 'float32',
 'product_description_lenght': 'float32',
 'product_photos_qty': 'float32',
 'product_weight_g': 'float32',
 'product_length_cm': 'float32',
 'product_height_cm': 'float32',
 'product_width_cm': 'float32',
 'seller_zip_code_prefix': 'float32',
 'geolocation_zip_code_prefix': 'float32',
 'geolocation_lat': 'float32',
 'geolocation_lng': 'float32'}

## Variables catégorielles
Pour réduire l'empreinte mémoire des données nous convertissons les données catégorielles avec le dtype 'category'.

In [14]:
# %%flake8  # checking PEP8 compliance
from pandas.api.types import CategoricalDtype

# List of categorical features
columns_to_convert_to_categorical = [
    'customer_id',
    'customer_unique_id',
    'customer_zip_code_prefix',
    'customer_city',
    'customer_state',
    'geolocation_zip_code_prefix',
    'geolocation_city',
    'geolocation_state',
    'order_id',
    'customer_id',
    'order_status',
    'order_id',
    'order_item_id',
    'product_id',
    'seller_id',
    'order_id',
    'payment_type',
    'review_id',
    'order_id',
    'product_id',
    'product_category_name',
    'seller_id',
    'seller_zip_code_prefix',
    'seller_city',
    'seller_state',
    'product_category_name',
    'product_category_name_english',
]

# Define dtype as 'category' for above features
for column_name in columns_to_convert_to_categorical:
    data[column_name] = data[column_name].astype('category')

# Define 'review_score' as ordinal categorical data
# cat_type = CategoricalDtype(categories=[1, 2, 3, 4, 5], ordered=True)
# data['review_score'] = data['review_score'].astype(cat_type)

## Variables temporelles
Pour réduire l'empreinte mémoire des données nous convertissons les données temporelle avec le dtype 'datetime'.

In [15]:
# %%flake8  # checking PEP8 compliance

# List of columns to convert to datetime
columns_to_convert_to_datetime = [
    'order_purchase_timestamp',
    'order_approved_at',
    'order_delivered_carrier_date',
    'order_delivered_customer_date',
    'order_estimated_delivery_date',
    'shipping_limit_date',
    'review_creation_date',
    'review_answer_timestamp',
]

# Converting dtype to datetime features
for column_name in columns_to_convert_to_datetime:
    data[column_name] = pd.to_datetime(data[column_name])

# [[⇑]](#tableofcontents) **Sélection des variables** <a name="selection"></a>

Pour alléger la base de données, nous allons supprimer les colonnes qui n'apportent pas d'informations sur les clients.

In [16]:
print(data['customer_id'].nunique())
print(data['customer_unique_id'].nunique())
len(data)

99441
96096


119151

L'identifiant unique 'customer_unique_id' semble davantage spécifique. Cela signifie uqe plusieurs 'customer_id' peuvent être rattachés à un même 'customer_unique_id'.

In [17]:
# %%flake8  # checking PEP8 compliance

# Dropping useless columns
data = data.drop(
    columns=[
        'customer_id',  # duplicate information with 'customer_unique_id'
        'customer_city',  # duplicate information with zip code
        'customer_state',  # duplicate information with zip code
        'geolocation_zip_code_prefix',  # duplicate information with zip code
        'geolocation_city',  # duplicate information with zip code
        'geolocation_state',  # duplicate information with zip code
        'order_approved_at',  # internal information of the company
        'order_delivered_carrier_date',  # internal information of the company
        'order_estimated_delivery_date',  # internal information of the company
        'seller_id',  # information about seller, not customer
        'shipping_limit_date',  # information for seller, not customer
        'product_category_name',  # duplicates product_category_name_english
        'seller_zip_code_prefix',  # information about seller, not customer
        'seller_city',  # information about seller, not customer
        'seller_state',  # information about seller, not customer
    ])

In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 119151 entries, 0 to 119150
Data columns (total 30 columns):
customer_unique_id               119151 non-null category
customer_zip_code_prefix         119151 non-null category
order_id                         119151 non-null category
order_status                     119151 non-null category
order_purchase_timestamp         119151 non-null datetime64[ns]
order_delivered_customer_date    115730 non-null datetime64[ns]
payment_sequential               119148 non-null float32
payment_type                     119148 non-null category
payment_installments             119148 non-null float32
payment_value                    119148 non-null float32
review_id                        119151 non-null category
review_score                     119151 non-null int8
review_comment_title             14189 non-null object
review_comment_message           51250 non-null object
review_creation_date             119151 non-null datetime64[ns]
review_answer_

# [[⇑]](#tableofcontents) **Ingénierie des caractéristiques** (*Features engineering*) <a name="engineering"></a>

## *Features engineering* au niveau des articles commandés (table *data*)

**Au niveau des achats** (table *data*)
* binning pour : 
    * product_name_lenght
    * product_description_lenght
    * product_photos_qty (aucune, 1-5, 5+)
    * product_weight_g

In [19]:
data.tail()

,customer_unique_id,customer_zip_code_prefix,order_id,order_status,order_purchase_timestamp,order_delivered_customer_date,payment_sequential,payment_type,payment_installments,payment_value,...,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english,geolocation_lat,geolocation_lng
119146,1a29b476fee25c95fbafc67c5ac95cf8,3937,6760e20addcf0121e9d58f2f1ff14298,delivered,2018-04-07 15:48:17,2018-04-13 20:06:37,1.0,credit_card,6.0,88.779999,...,51.0,489.0,2.0,611.0,22.0,22.0,23.0,books_general_interest,-23.587900,-46.501831
119147,d52a67c98be1cf6a5c84435bd38d095d,6764,9ec0c8947d973db4f4e8dcf1fbfa8f1b,delivered,2018-04-04 08:20:22,2018-04-11 18:54:45,1.0,credit_card,3.0,129.059998,...,51.0,1193.0,1.0,1211.0,25.0,24.0,22.0,sports_leisure,-23.612295,-46.765785
119148,e9f50caf99f032f0bf3c55141f019d99,60115,fed4434add09a6f332ea398efd656a5c,delivered,2018-04-08 20:11:50,2018-05-09 19:03:15,1.0,credit_card,5.0,56.040001,...,60.0,575.0,1.0,870.0,25.0,20.0,18.0,health_beauty,-3.744128,-38.510860
119149,73c2643a0a458b49f58cea58833b192e,92120,e31ec91cea1ecf97797787471f98a8c2,delivered,2017-11-03 21:08:33,2017-11-16 19:58:39,1.0,credit_card,2.0,711.070007,...,59.0,452.0,1.0,710.0,19.0,13.0,14.0,watches_gifts,-29.956390,-51.167614
119150,84732c5050c01db9b23e19ba39899398,6703,28db69209a75e59f20ccbb5c36a20b90,delivered,2017-12-19 14:27:23,2017-12-26 18:42:36,1.0,credit_card,1.0,21.770000,...,54.0,933.0,1.0,50.0,16.0,5.0,11.0,perfumery,-23.593985,-46.910591


In [20]:
# %%flake8  # checking PEP8 compliance

# Adding a feature for the volume
data['product_volume_cm3'] = \
    data['product_length_cm'] * data['product_height_cm'] * data['product_width_cm']

data = data.drop(
    columns=[
        'product_length_cm',
        'product_height_cm',
        'product_width_cm',
    ])

In [21]:
# %%flake8  # checking PEP8 compliance

# Convert 'review_comment_title' as boolean feature
data['review_comment_title_bool'] = data['review_comment_title'].notnull()
data['review_comment_title_bool'].value_counts() / len(data)
data = data.drop(columns=['review_comment_title'])

In [22]:
# %%flake8  # checking PEP8 compliance

# Convert 'review_comment_title' as boolean feature
data['review_comment_message_bool'] = data['review_comment_message'].notnull()
data['review_comment_message_bool'].value_counts() / len(data)
data = data.drop(columns=['review_comment_message'])

In [23]:
# %%flake8  # checking PEP8 compliance

# Converting 'payment_sequential' as a boolean feature 'payment_sequential'
display(data['payment_sequential'].value_counts().head())
data['single_payment_sequential'] = (data['payment_sequential'] == 1)
data = data.drop(columns=['payment_sequential'])
data['single_payment_sequential'].value_counts() / len(data)

1.0    114007
2.0      3415
3.0       658
4.0       322
5.0       194
Name: payment_sequential, dtype: int64

True     0.956828
False    0.043172
Name: single_payment_sequential, dtype: float64

In [24]:
# %%flake8  # checking PEP8 compliance

# Replacing the column 'freight_value' by a feature 'freight_value_percentage'
data['freight_value_percentage'] = data['freight_value'] / data['price']
data = data.drop(columns=['freight_value'])

In [25]:
# %%flake8  # checking PEP8 compliance

# Converting the column 'order_status' as a boolean feature 'order_delivered'
display(data['order_status'].value_counts())
data['order_delivered'] = (data['order_status'] == 'delivered')
data = data.drop(columns=['order_status'])
data['order_delivered'].value_counts() / len(data)

delivered      115731
shipped          1256
canceled          750
unavailable       652
invoiced          378
processing        376
created             5
approved            3
Name: order_status, dtype: int64

True     0.971297
False    0.028703
Name: order_delivered, dtype: float64

In [26]:
data['payment_type'].value_counts()

credit_card    87784
boleto         23190
voucher         6465
debit_card      1706
not_defined        3
Name: payment_type, dtype: int64

In [27]:
# Échelonnage des paiements
data['payment_installments'].value_counts()

1.0     59447
2.0     13840
3.0     11889
4.0      8073
10.0     6978
5.0      6097
8.0      5122
6.0      4674
7.0      1848
9.0       739
12.0      168
15.0       93
18.0       38
24.0       34
11.0       26
20.0       21
13.0       19
14.0       16
17.0        8
16.0        7
21.0        6
0.0         3
22.0        1
23.0        1
Name: payment_installments, dtype: int64

In [28]:
data['payment_type'].value_counts()

credit_card    87784
boleto         23190
voucher         6465
debit_card      1706
not_defined        3
Name: payment_type, dtype: int64

In [29]:
# %%flake8  # checking PEP8 compliance
import pandas as pd
from datetime import timedelta

# Adding a column 'review period after delivery'
data['review period'] = \
    pd.to_timedelta(data['review_answer_timestamp']
                    - data['order_delivered_customer_date'])

data = data.drop(columns=['review_creation_date'])
data = data.drop(columns=['review_answer_timestamp'])

In [30]:
# Adding a column for 'delivery period', replacing the 'order_delivered_customer_date'
data['delivery period'] = pd.to_timedelta(data['order_delivered_customer_date'] - data['order_purchase_timestamp'])
data = data.drop(columns=['order_delivered_customer_date'])

In [31]:
pd.set_option("display.max_columns", 1000) # Maximum number of columns
data.tail()

,customer_unique_id,customer_zip_code_prefix,order_id,order_purchase_timestamp,payment_type,payment_installments,payment_value,review_id,review_score,order_item_id,product_id,price,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_category_name_english,geolocation_lat,geolocation_lng,product_volume_cm3,review_comment_title_bool,review_comment_message_bool,single_payment_sequential,freight_value_percentage,order_delivered,review period,delivery period
119146,1a29b476fee25c95fbafc67c5ac95cf8,3937,6760e20addcf0121e9d58f2f1ff14298,2018-04-07 15:48:17,credit_card,6.0,88.779999,36e2cdbaa9f639b57c53b37ac798fee8,4,1.0,ccb4503d9d43d245d3b295d0544f988b,74.900002,51.0,489.0,2.0,611.0,books_general_interest,-23.587900,-46.501831,11132.0,True,False,True,0.185314,True,14 days 15:08:27,6 days 04:18:20
119147,d52a67c98be1cf6a5c84435bd38d095d,6764,9ec0c8947d973db4f4e8dcf1fbfa8f1b,2018-04-04 08:20:22,credit_card,3.0,129.059998,b273b431c3aedb4eed18643309652940,5,1.0,9ede6b0570a75a4b9de4f383329f99ee,114.900002,51.0,1193.0,1.0,1211.0,sports_leisure,-23.612295,-46.765785,13200.0,False,False,True,0.123238,True,4 days 15:41:20,7 days 10:34:23
119148,e9f50caf99f032f0bf3c55141f019d99,60115,fed4434add09a6f332ea398efd656a5c,2018-04-08 20:11:50,credit_card,5.0,56.040001,fa4f16891e6b2edd1354668d07f5648b,1,1.0,7a5d2e1e131a860ae7d18f6fffa9d689,37.000000,60.0,575.0,1.0,870.0,health_beauty,-3.744128,-38.510860,9000.0,True,True,True,0.514595,True,-2 days +06:37:52,30 days 22:51:25
119149,73c2643a0a458b49f58cea58833b192e,92120,e31ec91cea1ecf97797787471f98a8c2,2017-11-03 21:08:33,credit_card,2.0,711.070007,0bcdc9e450ea500811a8d39ee993cd47,5,1.0,f819f0c84a64f02d3a5606ca95edd272,689.000000,59.0,452.0,1.0,710.0,watches_gifts,-29.956390,-51.167614,3458.0,False,False,True,0.032032,True,3 days 03:35:39,12 days 22:50:06
119150,84732c5050c01db9b23e19ba39899398,6703,28db69209a75e59f20ccbb5c36a20b90,2017-12-19 14:27:23,credit_card,1.0,21.770000,d95d701d062d2bb88041768a63aa750d,5,1.0,017692475c1c954ff597feda05131d73,13.990000,54.0,933.0,1.0,50.0,perfumery,-23.593985,-46.910591,880.0,False,True,True,0.556112,True,1 days 04:38:30,7 days 04:15:13


# [[⇑]](#tableofcontents) **Lignes dupliquées & valeurs manquantes** <a name="missing"></a>

In [32]:
nb_rows_before_dropping_duplicates = len(data)
data = data.drop_duplicates()

print("Number of dropped duplicated rows:", nb_rows_before_dropping_duplicates - len(data))
print("Number of remaining rows:", len(data))

Number of dropped duplicated rows: 608
Number of remaining rows: 118543


In [33]:
# Displays the global filling rate of the DataFrame
functions_p5.global_filling_rate(data)

DataFrame has 118543 rows and 27 columns.

Global filling rate of the DataFrame: 99.30%
Missing values in the DataFrame: 22374 (0.70%)

Number of rows with missing values: 5327 (4.49%)
Number of columns with missing values: 17 (62.96%)


In [34]:
# Display filling rate and number of missing values for each feature
for feature in data.columns:
    functions_p5.feature_filling_rate(data, feature, missing_only=True)

The filling rate of the column 'payment_type' is: 100.00%        (3 missing values)
The filling rate of the column 'payment_installments' is: 100.00%        (3 missing values)
The filling rate of the column 'payment_value' is: 100.00%        (3 missing values)
The filling rate of the column 'order_item_id' is: 99.31%        (815 missing values)
The filling rate of the column 'product_id' is: 99.31%        (815 missing values)
The filling rate of the column 'price' is: 99.31%        (815 missing values)
The filling rate of the column 'product_name_lenght' is: 97.89%        (2496 missing values)
The filling rate of the column 'product_description_lenght' is: 97.89%        (2496 missing values)
The filling rate of the column 'product_photos_qty' is: 97.89%        (2496 missing values)
The filling rate of the column 'product_weight_g' is: 99.30%        (834 missing values)
The filling rate of the column 'product_category_name_english' is: 97.87%        (2521 missing values)
The filling rat

## Imputation des valeurs manquantes

## Suppression des lignes contenant des valeurs manquantes

In [35]:
# Delete rows with missing values
nb_rows_before_dropping_missing = len(data)
mask = data.isnull().any(axis=1)
data = data[~mask]
print("Number of deleted rows:", nb_rows_before_dropping_missing - len(data))
print("Number of remaining rows:", len(data))

Number of deleted rows: 5327
Number of remaining rows: 113216


# [[⇑]](#tableofcontents) **Aggrégations par clients** (*groupby*) <a name="aggregation"></a>

**Au niveau de l'aggrégation par client** (*groupby*)

* % des commandes (pas articles) livrées
* récupérer la localisation (latitude, longitude)

In [36]:
pd.set_option("display.max_columns", 1000) # Maximum number of columns
data.head()

,customer_unique_id,customer_zip_code_prefix,order_id,order_purchase_timestamp,payment_type,payment_installments,payment_value,review_id,review_score,order_item_id,product_id,price,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_category_name_english,geolocation_lat,geolocation_lng,product_volume_cm3,review_comment_title_bool,review_comment_message_bool,single_payment_sequential,freight_value_percentage,order_delivered,review period,delivery period
0,861eff4711a542e4b93843c6dd7febb0,14409,00e7ee1b050b8499577073aeb2a297a1,2017-05-16 15:05:35,credit_card,2.0,146.869995,88b8b52d46df026a9d1ad2136a59b30b,4,1.0,a9516a079e37a9c9c36b9b78b10169e8,124.989998,41.0,1141.0,1.0,8683.0,office_furniture,-20.509897,-47.397865,107136.0,False,False,True,0.175054,True,5 days 11:59:05,8 days 19:30:00
1,290c77bc529b7ac935b93aa66c333dc3,9790,29150127e6685892b6eab3eec79f59c7,2018-01-12 20:48:24,credit_card,8.0,335.480011,02fc48a9efa3e3d0f1a8ea26507eeec3,5,1.0,4aa6014eceb682077f9dc4bffebc05b0,289.000000,43.0,1002.0,3.0,10150.0,housewares,-23.726852,-46.545746,53400.0,False,False,True,0.160830,True,12 days 10:02:10,16 days 15:52:55
2,060e732b5b29e8181a18229c7b0b2b5e,1151,b2059ed67ce144a36e2aa97d2c9e9ad2,2018-05-19 16:07:45,credit_card,7.0,157.729996,5ad6695d76ee186dc473c42706984d87,5,1.0,bd07b66896d6f1494f5b86251848ced7,139.940002,55.0,955.0,1.0,8267.0,office_furniture,-23.527788,-46.660309,45968.0,False,False,True,0.127126,True,0 days 18:12:08,26 days 01:51:06
3,259dac757896d24d7702b9acbbff3f3c,8775,951670f92359f4fe4a63112aa7306eba,2018-03-13 16:06:38,credit_card,1.0,173.300003,059a801bb31f6aab2266e672cab87bc5,5,1.0,a5647c44af977b148e0a3a4751a09e2e,149.940002,48.0,1066.0,1.0,12160.0,office_furniture,-23.496929,-46.185352,79968.0,False,False,True,0.155796,True,5 days 02:32:22,14 days 23:57:47
4,345ecd01c38d18a9036ed96c73b8d066,13056,6b7d50bd145f6fc7f33cebabd7e49d0f,2018-07-29 09:51:30,credit_card,8.0,252.250000,8490879d58d6c5d7773f2739a03f089a,5,1.0,9391a573abe00141c56e38d84d7d5b3b,230.000000,61.0,407.0,1.0,5200.0,home_confort,-22.987223,-47.151073,23625.0,True,True,True,0.096739,True,7 days 05:04:04,11 days 11:04:18


In [37]:
# Initializing the data about customers
data_customers = dict_df['olist_customers_dataset']
data_customers.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [38]:
# Creating the groupby object by 'customer_unique_id'
data_gb = data.groupby(by='customer_unique_id')

In [39]:
# Defining aggregation function for strings
def agg_str(x): return {x.loc[row_id] for row_id in x.index}

In [40]:
# %%flake8  # checking PEP8 compliance

features_to_add = []

# Add a feature : total number of items (groupby customers)
features_to_add.append(data_gb.size()
                       .rename('nb_items'))

# Add a feature : total of different products ordered (groupby customers)
features_to_add.append(data_gb['product_id'].nunique()
                       .rename('nb_products'))

# Add a feature : number of orders (groupby customers)
features_to_add.append(data_gb['order_id'].nunique()
                       .rename('nb_orders'))

# Add a feature : total of prices (groupby customers)
features_to_add.append(data_gb['price'].sum()
                       .rename('sum_prices'))

# Add a feature : sum of payment_value (groupby customers)
features_to_add.append(data_gb['payment_value'].sum()
                       .rename('payment_sum'))

# Add a feature : max payment installments (groupby customers)
features_to_add.append(data_gb['payment_installments'].max()
                       .rename('max_payment_installements'))

# Add a feature : number of reviews (groupby customers)
features_to_add.append(data_gb['review_id'].nunique()
                       .rename('nb_reviews'))

# Add a feature : mean score for reviews (groupby customers)
features_to_add.append(data_gb['review_score'].mean()
                       .rename('mean_review_score'))

# Add a feature : count of reviews comments (groupby customers)
features_to_add.append(data_gb['review_comment_title_bool'].count()
                       .rename('review_comment_count'))

# Add a feature : count of reviews titles (groupby customers)
features_to_add.append(data_gb['review_comment_title_bool'].count()
                       .rename('review_title_count'))

# Add a feature : mean freight_value_percentage (groupby customers)
features_to_add.append(data_gb['freight_value_percentage'].mean()
                       .rename('mean_freight_value_percentage'))

# Add a feature : mean weight of products (groupby customers)
features_to_add.append(data_gb['product_weight_g'].mean()
                       .rename('mean_product_weight_g'))

# Add a feature : first order date (groupby customers)
features_to_add.append(data_gb['order_purchase_timestamp'].min()
                       .rename('first_order_purchase_timestamp'))

# Add a feature : last order date (groupby customers)
features_to_add.append(data_gb['order_purchase_timestamp'].max()
                       .rename('last_order_purchase_timestamp'))

# Adding a feature for 'set_product_categories'
features_to_add.append(data_gb['product_category_name_english'].agg(agg_str)
                       .rename('product_categories'))

# Adding a feature for 'set_product_categories'
features_to_add.append(data_gb['payment_type'].agg(agg_str)
                       .rename('payment_types'))

# Merging the new features to the DataFrame
for feature_series in features_to_add:
    data_customers = pd.merge(data_customers, feature_series, on='customer_unique_id')

data_customers.tail()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,nb_items,nb_products,nb_orders,sum_prices,payment_sum,max_payment_installements,nb_reviews,mean_review_score,review_comment_count,review_title_count,mean_freight_value_percentage,mean_product_weight_g,first_order_purchase_timestamp,last_order_purchase_timestamp,product_categories,payment_types
99436,17ddf5dd5d51696bb3d7c6291687be6f,1a29b476fee25c95fbafc67c5ac95cf8,3937,sao paulo,SP,1,1,1,74.900002,88.779999,6.0,1,4.0,1,1,0.185314,611.0,2018-04-07 15:48:17,2018-04-07 15:48:17,{books_general_interest},{credit_card}
99437,e7b71a9017aa05c9a7fd292d714858e8,d52a67c98be1cf6a5c84435bd38d095d,6764,taboao da serra,SP,1,1,1,114.900002,129.059998,3.0,1,5.0,1,1,0.123238,1211.0,2018-04-04 08:20:22,2018-04-04 08:20:22,{sports_leisure},{credit_card}
99438,5e28dfe12db7fb50a4b2f691faecea5e,e9f50caf99f032f0bf3c55141f019d99,60115,fortaleza,CE,1,1,1,37.000000,56.040001,5.0,1,1.0,1,1,0.514595,870.0,2018-04-08 20:11:50,2018-04-08 20:11:50,{health_beauty},{credit_card}
99439,56b18e2166679b8a959d72dd06da27f9,73c2643a0a458b49f58cea58833b192e,92120,canoas,RS,1,1,1,689.000000,711.070007,2.0,1,5.0,1,1,0.032032,710.0,2017-11-03 21:08:33,2017-11-03 21:08:33,{watches_gifts},{credit_card}
99440,274fa6071e5e17fe303b9748641082c8,84732c5050c01db9b23e19ba39899398,6703,cotia,SP,1,1,1,13.990000,21.770000,1.0,1,5.0,1,1,0.556112,50.0,2017-12-19 14:27:23,2017-12-19 14:27:23,{perfumery},{credit_card}


### Création d'une variable : % de commandes livrées

# [[⇑]](#tableofcontents) **Ingénierie des caractéristiques** aggrégées par client <a name="engineering_customers"></a>

In [41]:
data_customers.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,nb_items,nb_products,nb_orders,sum_prices,payment_sum,max_payment_installements,nb_reviews,mean_review_score,review_comment_count,review_title_count,mean_freight_value_percentage,mean_product_weight_g,first_order_purchase_timestamp,last_order_purchase_timestamp,product_categories,payment_types
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,1,1,1,124.989998,146.869995,2.0,1,4.0,1,1,0.175054,8683.0,2017-05-16 15:05:35,2017-05-16 15:05:35,{office_furniture},{credit_card}
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,1,1,1,289.000000,335.480011,8.0,1,5.0,1,1,0.160830,10150.0,2018-01-12 20:48:24,2018-01-12 20:48:24,{housewares},{credit_card}
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,1,1,1,139.940002,157.729996,7.0,1,5.0,1,1,0.127126,8267.0,2018-05-19 16:07:45,2018-05-19 16:07:45,{office_furniture},{credit_card}
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP,1,1,1,149.940002,173.300003,1.0,1,5.0,1,1,0.155796,12160.0,2018-03-13 16:06:38,2018-03-13 16:06:38,{office_furniture},{credit_card}
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP,1,1,1,230.000000,252.250000,8.0,1,5.0,1,1,0.096739,5200.0,2018-07-29 09:51:30,2018-07-29 09:51:30,{home_confort},{credit_card}


**Ingénierie des caractéristiques**
* binning payment_installments

In [42]:
# %%flake8  # checking PEP8 compliance

# Removing columns 'customer_id' and 'customer_unique_id'
data_customers = data_customers.drop(columns=[
    'customer_id',
    'customer_unique_id',
])

# Adding a feature for 'customer_active_seniority'
data_customers['customer_active_seniority'] = pd.to_timedelta(
    data_customers['last_order_purchase_timestamp']
    - data_customers['first_order_purchase_timestamp'])

# Adding a feature for 'percentage_review' (related to number of orders)
data_customers['percentage_review'] = \
    data_customers['nb_reviews'] / data_customers['nb_orders']
data_customers = data_customers.drop(columns=['nb_reviews'])

# Adding a feature for 'mean_period_between_orders'
data_customers['mean_period_between_orders'] = \
    data_customers['customer_active_seniority'] / data_customers['nb_orders']

# Adding a feature for 'mean_price_of_items'
data_customers['mean_price_of_items'] =\
    data_customers['sum_prices'] / data_customers['nb_items']

# Adding a feature for 'mean_price_of_orders'
data_customers['mean_price_of_orders'] = \
    data_customers['sum_prices'] / data_customers['nb_orders']

# Adding a feature for 'mean_nb_items_per_order'
data_customers['mean_nb_items_per_order'] = \
    data_customers['nb_items'] / data_customers['nb_orders']

In [43]:
data_customers.head()

,customer_zip_code_prefix,customer_city,customer_state,nb_items,nb_products,nb_orders,sum_prices,payment_sum,max_payment_installements,mean_review_score,review_comment_count,review_title_count,mean_freight_value_percentage,mean_product_weight_g,first_order_purchase_timestamp,last_order_purchase_timestamp,product_categories,payment_types,customer_active_seniority,percentage_review,mean_period_between_orders,mean_price_of_items,mean_price_of_orders,mean_nb_items_per_order
0,14409,franca,SP,1,1,1,124.989998,146.869995,2.0,4.0,1,1,0.175054,8683.0,2017-05-16 15:05:35,2017-05-16 15:05:35,{office_furniture},{credit_card},0 days,1.0,0 days,124.989998,124.989998,1.0
1,9790,sao bernardo do campo,SP,1,1,1,289.000000,335.480011,8.0,5.0,1,1,0.160830,10150.0,2018-01-12 20:48:24,2018-01-12 20:48:24,{housewares},{credit_card},0 days,1.0,0 days,289.000000,289.000000,1.0
2,1151,sao paulo,SP,1,1,1,139.940002,157.729996,7.0,5.0,1,1,0.127126,8267.0,2018-05-19 16:07:45,2018-05-19 16:07:45,{office_furniture},{credit_card},0 days,1.0,0 days,139.940002,139.940002,1.0
3,8775,mogi das cruzes,SP,1,1,1,149.940002,173.300003,1.0,5.0,1,1,0.155796,12160.0,2018-03-13 16:06:38,2018-03-13 16:06:38,{office_furniture},{credit_card},0 days,1.0,0 days,149.940002,149.940002,1.0
4,13056,campinas,SP,1,1,1,230.000000,252.250000,8.0,5.0,1,1,0.096739,5200.0,2018-07-29 09:51:30,2018-07-29 09:51:30,{home_confort},{credit_card},0 days,1.0,0 days,230.000000,230.000000,1.0


# [[⇑]](#tableofcontents) **Encodage des caractéristiques** <a name="encoding"></a>

Les algorithmes de *clustering* n'acceptant en entrée que des données numériques, nous allons encoder numériquement les données qui ne sont pas numériques (données catégorielles).

In [44]:
data_customers.dtypes

customer_zip_code_prefix                    int64
customer_city                              object
customer_state                             object
nb_items                                    int64
nb_products                                 int64
nb_orders                                   int64
sum_prices                                float32
payment_sum                               float32
max_payment_installements                 float32
mean_review_score                         float64
review_comment_count                        int64
review_title_count                          int64
mean_freight_value_percentage             float32
mean_product_weight_g                     float32
first_order_purchase_timestamp     datetime64[ns]
last_order_purchase_timestamp      datetime64[ns]
product_categories                         object
payment_types                              object
customer_active_seniority         timedelta64[ns]
percentage_review                         float64


### Encodage '*One-hot*' des données catégorielles nominales

In [45]:
# import libraries
import numpy as np
from sklearn.preprocessing import LabelBinarizer

# list the features to 'one-hot encode' (vectorization)
nominal_categorical_features = [
    'customer_zip_code_prefix',
    'customer_city',
    'customer_state',
    ]

# one-hot encode features
for feature in nominal_categorical_features:
    # create one-hot encoder
    one_hot = LabelBinarizer()
    # apply one-hot encoding
    feature_encoded = one_hot.fit_transform(data_customers[feature])
    # split the nd.array as list of vectors
    feature_encoded = np.split(feature_encoded, indices_or_sections=range(1,len(feature_encoded)), axis=0)
    # flatten the result
    feature_encoded = [vector.flatten() for vector in feature_encoded]
    # add a new column in the DataFrame
    data_customers[feature+'_ohe'] = feature_encoded
    # delete original columns
    data_customers = data_customers.drop(columns=[feature])

### Encodage 'Multiclass One-hot' des données catégorielles nominales

In [46]:
# import libraries
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer

# list the features to 'one-hot encode' (vectorization)
multilabel_categorical_features = [
    'product_categories',
    'payment_types',
    ]

# one-hot encode features
for feature in multilabel_categorical_features:
    # create one-hot encoder
    one_hot = MultiLabelBinarizer()
    # apply one-hot encoding
    feature_encoded = one_hot.fit_transform(data_customers[feature])
    # split the nd.array as list of vectors
    feature_encoded = np.split(feature_encoded, indices_or_sections=range(1,len(feature_encoded)), axis=0)
    # flatten the result
    feature_encoded = [vector.flatten() for vector in feature_encoded]
    # add a new column in the DataFrame
    data_customers[feature+'_ohe'] = feature_encoded
    # delete original columns
    data_customers = data_customers.drop(columns=[feature])

In [47]:
data_customers.head()

,nb_items,nb_products,nb_orders,sum_prices,payment_sum,max_payment_installements,mean_review_score,review_comment_count,review_title_count,mean_freight_value_percentage,mean_product_weight_g,first_order_purchase_timestamp,last_order_purchase_timestamp,customer_active_seniority,percentage_review,mean_period_between_orders,mean_price_of_items,mean_price_of_orders,mean_nb_items_per_order,customer_zip_code_prefix_ohe,customer_city_ohe,customer_state_ohe,product_categories_ohe,payment_types_ohe
0,1,1,1,124.989998,146.869995,2.0,4.0,1,1,0.175054,8683.0,2017-05-16 15:05:35,2017-05-16 15:05:35,0 days,1.0,0 days,124.989998,124.989998,1.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0]"
1,1,1,1,289.000000,335.480011,8.0,5.0,1,1,0.160830,10150.0,2018-01-12 20:48:24,2018-01-12 20:48:24,0 days,1.0,0 days,289.000000,289.000000,1.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0]"
2,1,1,1,139.940002,157.729996,7.0,5.0,1,1,0.127126,8267.0,2018-05-19 16:07:45,2018-05-19 16:07:45,0 days,1.0,0 days,139.940002,139.940002,1.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0]"
3,1,1,1,149.940002,173.300003,1.0,5.0,1,1,0.155796,12160.0,2018-03-13 16:06:38,2018-03-13 16:06:38,0 days,1.0,0 days,149.940002,149.940002,1.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0]"
4,1,1,1,230.000000,252.250000,8.0,5.0,1,1,0.096739,5200.0,2018-07-29 09:51:30,2018-07-29 09:51:30,0 days,1.0,0 days,230.000000,230.000000,1.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0]"


In [48]:
data_customers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99441 entries, 0 to 99440
Data columns (total 24 columns):
nb_items                          99441 non-null int64
nb_products                       99441 non-null int64
nb_orders                         99441 non-null int64
sum_prices                        99441 non-null float32
payment_sum                       99441 non-null float32
max_payment_installements         95136 non-null float32
mean_review_score                 95136 non-null float64
review_comment_count              99441 non-null int64
review_title_count                99441 non-null int64
mean_freight_value_percentage     95136 non-null float32
mean_product_weight_g             95136 non-null float32
first_order_purchase_timestamp    95136 non-null datetime64[ns]
last_order_purchase_timestamp     95136 non-null datetime64[ns]
customer_active_seniority         95136 non-null timedelta64[ns]
percentage_review                 95136 non-null float64
mean_period_between_orde

# [[⇑]](#tableofcontents) **Pandas profiling** <a name="profiling"></a>

In [ ]:
import pandas_profiling

pandas_profiling.ProfileReport(data_customers)

,nb_items,nb_products,nb_orders,sum_prices,payment_sum,max_payment_installements,mean_review_score,review_comment_count,review_title_count,mean_freight_value_percentage,mean_product_weight_g,first_order_purchase_timestamp,last_order_purchase_timestamp,customer_active_seniority,percentage_review,mean_period_between_orders,mean_price_of_items,mean_price_of_orders,mean_nb_items_per_order,customer_zip_code_prefix_ohe,customer_city_ohe,customer_state_ohe,product_categories_ohe,payment_types_ohe
0,1,1,1,124.989998,146.869995,2.0,4.0,1,1,0.175054,8683.0,2017-05-16 15:05:35,2017-05-16 15:05:35,0 days,1.0,0 days,124.989998,124.989998,1.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0]"
1,1,1,1,289.000000,335.480011,8.0,5.0,1,1,0.160830,10150.0,2018-01-12 20:48:24,2018-01-12 20:48:24,0 days,1.0,0 days,289.000000,289.000000,1.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0]"
2,1,1,1,139.940002,157.729996,7.0,5.0,1,1,0.127126,8267.0,2018-05-19 16:07:45,2018-05-19 16:07:45,0 days,1.0,0 days,139.940002,139.940002,1.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0]"
3,1,1,1,149.940002,173.300003,1.0,5.0,1,1,0.155796,12160.0,2018-03-13 16:06:38,2018-03-13 16:06:38,0 days,1.0,0 days,149.940002,149.940002,1.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0]"
4,1,1,1,230.000000,252.250000,8.0,5.0,1,1,0.096739,5200.0,2018-07-29 09:51:30,2018-07-29 09:51:30,0 days,1.0,0 days,230.000000,230.000000,1.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99436,1,1,1,74.900002,88.779999,6.0,4.0,1,1,0.185314,611.0,2018-04-07 15:48:17,2018-04-07 15:48:17,0 days,1.0,0 days,74.900002,74.900002,1.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0]"
99437,1,1,1,114.900002,129.059998,3.0,5.0,1,1,0.123238,1211.0,2018-04-04 08:20:22,2018-04-04 08:20:22,0 days,1.0,0 days,114.900002,114.900002,1.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0]"
99438,1,1,1,37.000000,56.040001,5.0,1.0,1,1,0.514595,870.0,2018-04-08 20:11:50,2018-04-08 20:11:50,0 days,1.0,0 days,37.000000,37.000000,1.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0]"
99439,1,1,1,689.000000,711.070007,2.0,5.0,1,1,0.032032,710.0,2017-11-03 21:08:33,2017-11-03 21:08:33,0 days,1.0,0 days,689.000000,689.000000,1.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0]"


# [[⇑]](#tableofcontents) **Sauvegarde** (méthode *feather*) <a name="save"></a>

In [52]:
# feather does not support serializing a non-default index
data_customers = data_customers.reset_index()

# Deleting index
data_customers = data_customers.drop(columns=["index"])

# Save the file (binary)
data_customers.to_feather("p5_data.ft")

ArrowNotImplementedError: ('Unsupported numpy type 22', 'Conversion failed for column customer_active_seniority with type timedelta64[ns]')